<a href="https://colab.research.google.com/github/BugPersonality/ML/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

Без использования merge!

In [2]:
trTypes = pd.read_csv(filepath_or_buffer = ".vscode/csv/tr_types.csv", sep = ';')
transactions = pd.read_csv(filepath_or_buffer = ".vscode/csv/transactions.csv", sep = ',')
trMccCode = pd.read_csv(filepath_or_buffer = ".vscode/csv/tr_mcc_codes.csv", sep = ';')
genderTrain = pd.read_csv(filepath_or_buffer = ".vscode/csv/gender_train.csv", sep = ',')

In [3]:
def isContainsATM(string):
    return string.str.contains("АТМ").bool()

def isContainsPOS(string):
    return string.str.contains("POS").bool()

sample = transactions["tr_type"].sample(n = 1000)
count = 0

for i in sample: 
    description = trTypes.query(f'tr_type == {i}')['tr_description']
    if isContainsATM(description) or isContainsPOS(description):
        count += 1

print(count/1000)

0.624


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [5]:
allTypes = transactions.groupby('tr_type').size()
topTenTypes = pd.DataFrame(allTypes.sort_values(ascending=False).head(10).reset_index(name = 'freq'))

for index, row in topTenTypes.iterrows():
    trTypeInRow = row['tr_type']
    descriptionInTrTypeDF = trTypes.query(f'tr_type == {trTypeInRow}')['tr_description'].to_string(index = False)
    print(row['freq'], end=" ")
    print(row['tr_type'], end=" ")
    print(descriptionInTrTypeDF)

1589734 1010 Покупка. POS ТУ СБ РФ
1040713 2010 Выдача наличных в АТМ Сбербанк России
970994 1030 Оплата услуги. Банкоматы СБ РФ
928154 1110 Покупка. POS ТУ Россия
691896 7070 Перевод на карту (с карты) через Мобильный банк...
376521 2370 Списание с карты на карту по операции <перевода...
258088 7010 Взнос наличных через АТМ (в своем тер.банке)
155200 7030 Перевод на карту (с карты) через АТМ (в предела...
117870 1100 Покупка. ТУ  Россия
96146 7071 Перевод на карту (с карты) через Мобильный банк...


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [6]:
positiveAmount = transactions.query('amount > 0').groupby(['customer_id'])['amount']
negativeAmount = transactions.query('amount < 0').groupby(['customer_id'])['amount']

customerWithMaxAmount = positiveAmount.sum().sort_values(ascending=False).head(1).reset_index(name = 'amount')
customerWithMinAmount = negativeAmount.sum().sort_values(ascending=True).head(1).reset_index(name = 'amount')

print(f"id: {customerWithMaxAmount['customer_id'][0]}, amount: {customerWithMaxAmount['amount'][0]}")
print(f"id: {customerWithMinAmount['customer_id'][0]}, amount: {customerWithMinAmount['amount'][0]}")
print(f"diff: {abs(customerWithMaxAmount['amount'][0] - customerWithMinAmount['amount'][0])}")

id: 48281795, amount: 8542244401.49
id: 48281795, amount: -8541879253.05
diff: 17084123654.54


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [7]:
for i in topTenTypes['tr_type']:
    firstMean = transactions.query(f'tr_type == {i}')['amount'].mean()
    firstMedian1 = transactions.query(f'tr_type == {i}')['amount'].median()
    print(f"type: {i}, mean: {firstMean}, median: {firstMedian1}")

customerIndex = customerWithMinAmount['customer_id'][0]
secondMean = transactions.query(f'customer_id == {customerIndex}')['amount'].mean()
secondMedian = transactions.query(f'customer_id == {customerIndex}')['amount'].median()

print(f"customer_id: {customerIndex}, mean: {secondMean}, median: {secondMedian}")

type: 1010, mean: -17056.263410998316, median: -6733.26
type: 2010, mean: -135584.0633514523, median: -44918.32
type: 1030, mean: -4700.005100134502, median: -2245.92
type: 1110, mean: -26676.99850752138, median: -9365.47
type: 7070, mean: 100736.4855054083, median: 14284.02
type: 2370, mean: -197634.81372560895, median: -43795.36
type: 7010, mean: 259527.36668407678, median: 93205.5
type: 7030, mean: 135919.5084630155, median: 22459.16
type: 1100, mean: -39390.9236215322, median: -8983.66
type: 7071, mean: 137771.55287375452, median: 32341.19
customer_id: 48281795, mean: 231.5462523778226, median: -4491831.54


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [8]:
transactions = pd.merge(transactions, genderTrain, how='left')
transactions = pd.merge(transactions, trMccCode, how='inner')
transactions = pd.merge(transactions, trTypes, how='inner')

print(transactions.shape)

(6846580, 9)


## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [9]:
zeroSpentMean= transactions.query('gender == 0.0').query('amount < 0')['amount'].mean()
oneSpentMean = transactions.query('gender == 1.0').query('amount < 0')['amount'].mean()

print(f"Spent diff: {abs(zeroSpentMean - oneSpentMean)}")

zeroIncomeMean = transactions.query('gender == 0.0').query('amount > 0')['amount'].mean()
oneIncomeMean = transactions.query('gender == 1.0').query('amount > 0')['amount'].mean()

print(f"Income diff: {abs(zeroIncomeMean - oneIncomeMean)}")

Spent diff: 24748.03831067322
Income diff: 65084.17392010856


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 5 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [10]:
zeroSpentByMccCode = transactions.query('gender == 0.0').query('amount < 0').groupby('mcc_code')['amount'].size().sort_values(ascending=False)
oneSpentByMccCode = transactions.query('gender == 1.0').query('amount < 0').groupby('mcc_code')['amount'].size().sort_values(ascending=False)

zeroSpentByMccCode = zeroSpentByMccCode.head(len(zeroSpentByMccCode)).reset_index(name = 'amount')
oneSpentByMccCode = oneSpentByMccCode.head(len(oneSpentByMccCode)).reset_index(name = 'amount1')

print(zeroSpentByMccCode.head())
print(oneSpentByMccCode.head())

merged = pd.merge(zeroSpentByMccCode, oneSpentByMccCode, on = 'mcc_code', how = 'inner')
merged['diff'] = (abs(merged['amount'] - merged['amount1']))
merged = merged.sort_values(by = ['diff'], ascending=False).head()

print(merged)


   mcc_code  amount
0      6011  337395
1      4814  314319
2      5411  287538
3      4829  172622
4      5499   99229
   mcc_code  amount1
0      6011   320548
1      5411   251435
2      4814   234077
3      4829   158589
4      5499    89199
   mcc_code  amount  amount1   diff
1      4814  314319   234077  80242
9      5541   21243    58945  37702
2      5411  287538   251435  36103
5      5912   48346    27809  20537
0      6011  337395   320548  16847


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [11]:
transactions[['index', 'hour']] = transactions['tr_datetime'].str.split(" ", expand = True)
transactions['hour'] = transactions['hour'].str.slice(0, 2)
transactions['hour'] = pd.to_datetime(transactions['hour'], format = '%H')

In [12]:
one = transactions.query('amount < 0').query('gender == 1.0').groupby(['hour']).size().reset_index(name = 'count').head(6)
zero = transactions.query('amount < 0').query('gender == 0.0').groupby(['hour']).size().reset_index(name = 'count').head(6)

print(f"gender: 1, count: {one['count'].sum()}")
print(one)
print(f"gender: 1, count: {zero['count'].sum()}")
print(zero)


gender: 1, count: 309180
                 hour   count
0 1900-01-01 00:00:00  248459
1 1900-01-01 01:00:00    8954
2 1900-01-01 02:00:00    8862
3 1900-01-01 03:00:00   10485
4 1900-01-01 04:00:00   13656
5 1900-01-01 05:00:00   18764
gender: 1, count: 273498
                 hour   count
0 1900-01-01 00:00:00  205533
1 1900-01-01 01:00:00    8741
2 1900-01-01 02:00:00    9031
3 1900-01-01 03:00:00   11507
4 1900-01-01 04:00:00   15778
5 1900-01-01 05:00:00   22908
